# Άσκηση 3: Βελτιστοποίηση συναρτήσεων με Γενετικούς Αλγόριθμους
v1.0, 08/01/2019

<img src="http://infinity77.net/global_optimization/_images/Alpine01.png" alt="Alpine01" style="width: 500px;"/>

Στόχος της άσκησης είναι η βελτιστοποίηση συναρτήσεων χωρίς παραγώγους (derivative free optimization) με χρήση Γενετικών Αλγόριθμων μέσω της βιβλιοθήκης DEAP. Η βελτιστοποίηση χωρίς παραγώγους είναι ιδιαίτερα χρήσιμη σε περιπτώσεις όπου η αντικειμενική συνάρτηση $f$ δεν είναι διαθέσιμη (black-box optimization) ή σε περίπτωσεις που είναι, είναι δύσκολο ή μη πρακτικό να υπολογιστούν οι παράγωγοί της. Για παράδειγμα η  $f$ μπορεί να μην είναι διαφορίσιμη, η παραγώγιση της μπορεί να είναι δύσκολη ή να απαιτεί πολύ χρόνο,  η $f$ να περιέχει θόρυβο έτσι ώστε οι μέθοδοι που βασίζονται σε απειροστικές διαφορές να μην είναι αποτελεσματικές ή να υπάρχουν πολλά τοπικά βέλτιστα μεταξύ άλλων. 

Για περισσότερα πάνω στη βελτιστοποίηση χωρίς παραγώγους και τις εφαρμογές της, μπορείτε να συμβουλευτείτε το ["Gradient Free Optimization"](http://adl.stanford.edu/aa222/lecture_notes_files/chapter6_gradfree.pdf). 

Οφείλουμε εδώ να αναφέρουμε ότι κάποιες από τις συναρτήσεις που θα μελετήσουμε με ΓΑ μπορούν να βελτιστοποιηθούν πολύ ικανοποιητικά, ειδικά σε μικρές διαστάσεις, και με κλασικές αριθμητικές μεθόδους.

## Συναρτήσεις προς βελτιστοποίηση

Σε κάθε ομάδα αντιστοιχεί μία μη-κλιμακούμενη συνάρτηση (non-scalable, δηλαδή ορίζεται για συγκεκριμένες διαστάσεις του πεδίου ορισμού) και μία κλιμακούμενη (scalable, μπορεί να οριστεί για οποιαδήποτε διάσταση D). 

Καμία ομάδα δεν έχει τις ίδιες συναρτήσεις συνολικά με κάποια άλλη.

 Στο [“Ομάδες - Συναρτήσεις”](https://docs.google.com/spreadsheets/d/1g9otvvm1GIALF5c3Fsf45tULdoDZAmXnjmXyuW4sbro/edit?usp=sharing) θα βρείτε έναν αριθμό για τη μη-κλιμακούμενη συνάρτηση και έναν αριθμό για την κλιμακούμενη συνάρτηση που αντιστοιχούν στην ομάδα σας.
 
Θα βρείτε σε ποια συνάρτηση αντιστοιχεί ο κάθε αριθμός στο paper [“Α Literature Survey of Benchmark Functions For Global Optimization Problems”](https://goo.gl/qAhxNf), στο Section 3 (σελ. 5 και εξής). 

**Προσοχή:** ορισμένες συναρτήσεις όπως αναγράφονται στο paper έχουν μικρά (εώς μεσαία) λάθη ή χρειάζονται κάποια επεξήγηση. Έχουμε συγκεντρώσει τα errata που έχουμε ανακαλύψει μέχρι τώρα στο έγγραφο ["Errata στους ορισμούς συναρτήσεων"](https://drive.google.com/open?id=1MrRpq8q_hDfBZVw1CHBzC7t4biQkAftu3LEWXJMf6s0), παρακαλούμε συμβουλευτείτε το **απαραίτητα** για να δείτε αν αφορά σε κάποια από τις συναρτήσεις σας.

Οι ομάδες που έχουν μη-κλιμακούμενες συναρτήσεις με κωδικούς Α01-Α10 ή κλιμακούμενες συναρτήσεις με κωδικούς B01-B10 θα βρουν τους ορισμούς που τους αντιστοιχούν στο συμπληρωματικό έγγραφο ["Πρόσθετοι ορισμοί συναρτήσεων"](https://docs.google.com/document/d/1F-YJ5Ripo0N5ZAXCS52eGqNVB9RF6ZTvjnOh5ICHfss/edit?usp=sharing).

Όλες οι συναρτήσεις είναι προς **ελαχιστοποίηση**.

In [3]:
from math import cos, sin, pi

In [5]:
def f(x1, x2):
    return x1^2 + 2*x2^2 - 0.3*cos(3*pi*x1)*cos(4*pi*x2) + 0.3


## Επιλογή κατάλληλου Γενετικού Αλγόριθμου

Ο κάθε Γενετικός Αλγόριθμος χαρακτηρίζεται α) από τους γενετικούς τελεστές του δηλαδή τον ορισμό της διασταύρωσης, μετάλλαξης και επιλογής και β) από τη στρατηγική εξέλιξης που καθορίζει την εξέλιξη του πληθυσμού μέσω της διαδικασίας επιλογής, καθώς και των  πιθανοτήτων διασταύρωσης και μετάλλαξης.

Όπως πολύ συχνά ισχύει σε προβλήματα Μηχανικής Μάθησης, η επιλογή του κατάλληλου αλγόριθμου εξαρτάται από το κάθε πρόβλημα και τα δεδομένα του. Συνεπώς για κάθε συνάρτηση θα πρέπει να δοκιμαστούν διαφορετικοί συνδυασμοί τελεστών και στρατηγικών.

Θα χρησιμοποιήσουμε ως γονίδια πραγματικούς αριθμούς και ως χρωμοσώματα λίστες γονιδίων μήκους ίσο με τη διάσταση του πεδίου ορισμού της συνάρτησης.



### Αξιολόγηση Γενετικών Αλγόριθμων

Μπορούμε να πούμε ότι ένας ΓΑ είναι πιθανόν να έχει καλύτερα χαρακτηριστικά για ένα δεδομένο πρόβλημα αν:
- βρίσκει τη λύση (βέλτιστη τιμή) με μεγαλύτερη συχνότητα
- βρίσκει τη βέλτιστη λύση με μικρότερο αριθμό αποτιμήσεων της συνάρτησης καταλληλότητας




#### Απόλυτα κριτήρια
Πρακτικά, για να αξιολογήσουμε τους αλγόριθμους της άσκησης θα φτιάξουμε μια συνάρτηση που θα αναλύει τα logbooks του αλγόριθμου ως εξής:  

Για ένα μέγιστο αριθμό γύρων (MAX_ROUNDS)  
&nbsp;&nbsp;&nbsp;&nbsp;Για ένα μέγιστο αριθμό γενεών (MAX_GENS)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Για κάθε ΓΑ με δεδομένους γενετικούς τελεστές και στρατηγική εξέλιξης υπολογίζουμε τα εξής μεγέθη:

 
- **avg.min:** ο μέσος όρος της ελάχιστης τιμής που πέτυχε για όλους τους γύρους. Επί της ουσίας αυτό είναι το πιο σημαντικό και απόλυτο κριτήριο της επίδοσης του αλγόριθμου.
- **avg.evals:** o μέσος όρος των συνολικών αποτιμήσεων (άθροιση όλων των γενεών) που έκανε για όλους τους γύρους.
- **avg.time:** ο μέσος χρόνος εκτέλεσης των γύρων. 



#### Σχετικά κριτήρια

Τα προηγούμενα κριτήρια αποτελούν τα απόλυτα μέτρα της επίδοσης του αλγόριθμου.  Ωστόσο, επειδή δεν είναι εφικτό για κάθε συνάρτηση να αναμένουμε ότι σε λογικά χρονικά διαστήματα (ή και γενικά) θα βρούμε την ακριβή βέλτιστη λύση, ορίζουμε ως επιτυχία την ποσότητα **GOAL + DELTA**, όπου GOAL το ελάχιστο της συνάρτησης (δίνεται στο pdf ή στον ορισμό της συναρτησής σας) και DELTA μια θετική ποσότητα που θα ορίζουμε εμείς κάθε φορά (διαφορετική για κάθε συνάρτηση και διαφορετική ανάλογα του πόσο κοντά στη λύση θέλουμε να φτάσουμε). Με βάση την ποσότητα GOAL + DELTA ορίζουμε επιπλέον τα εξής μεγέθη:
- **successes:** αριθμός φορών που o ΓΑ βρήκε τιμή μικρότερη του GOAL + DELTA. Προφανώς το βέλτιστο είναι MAX_ROUNDS.
- **success avg. gen:** o μέσος όρος των γενεών που χρειάστηκε ο αλγόριθμος για να πέσει κάτω από GOAL + DELTA, για τους γύρους με επιτυχία. Ουσιαστικά μας ενδιαφέρει η πρώτη γενιά που πετυχαίνει τιμή κάτω από GOAL + DELTA. Σε περίπτωση που δεν πετυχαίνει αυτό το στόχο καμία γενιά, θέτουμε ‘None’ σε αυτή την ποσότητα.
- **success avg. min:** ο μέσος όρος των ελαχίστων των πρώτων επιτυχημένων γενεών για τους γύρους με επιτυχία. Παρόμοια, θέτουμε ‘None’ σε περίπτωση αποτυχίας.
- **success avg. evals:** ο μέσος όρος αποτιμήσεων που χρειάστηκαν για να πάρουμε την πρώτη τιμή κάτω από GOAL + DELTA. Και πάλι  θέτουμε ‘None’ σε περίπτωση αποτυχίας.

Πρόκειται ουσιαστικά για ένα σύνολο πιο "χαλαρών" κριτηρίων βελτιστοποίησης (λόγω του DELTA) που όμως θα μας βοηθήσουν να προσδιορίσουμε ευκολότερα τις υπερπαραμέτρους του τελικού γενετικού αλγόριθμου.




### Βοηθητική συνάρτηση εκτύπωσης υπερπαραμέτρων και απόδοσης (σχετικής και απόλυτης)

Φτιάχνουμε τέλος μια βοηθητική συνάρτηση που τυπώνει συγκεντρωτικά τα αποτελέσματα ως εξής (η επεξήγηση για τις δύο πρώτες κολόνες στο ακριβώς επόμενο section):

| operators                      | strategy                      | successes | s.avg.min | s.avg.evals | s.avg.gens | avg.evals | avg.min   | avg.time |
|--------------------------------|-------------------------------|-----------|-----------|-------------|------------|-----------|-----------|----------|
|CrossoverLow,MutationLow,SelectionLow | eaSimple 200 0.8 0.2          | 1         | 8.508e-42 | 16487       | 96         | 17014     | 9.115e-34 | 1.898    |
| CrossoverHigh,MutationLow,SelectionLow | eaMuPlusLambda 50 150 0.8 0.2 | 1         | 3.885e-41 | 14750       | 98         | 15050     | 1.631e-21 | 1.591    



## Μέρος 1. Βελτιστοποίηση μη κλιμακούμενης συνάρτησης



### Εύρεση βέλτιστου συνδυασμού τελεστών - στρατηγικής



#### Γενετικοί τελεστές

Αρχικά θα δοκιμάσουμε δύο διαφορετικούς τελεστές διασταύρωσης και δύο διαφορετικούς τελεστές μετάλλαξης της επιλογής μας, με δύο διαφορετικές τιμές υπερ-παραμέτρων για τον καθένα (μία χαμηλή-low και μία υψηλή-high - εάν κάποιος τελεστής έχει περισσότερες υπερπαραμέτρους απλά θεωρήστε δύο συνδυασμούς των τιμών τους, ο συνδυασμός high να οδηγεί σε μεγαλύτερη αλλαγή του γενετικού υλικού και αντίστροφα για τον συνδυασμό low). Θα δοκιμάσουμε επίσης δύο τιμές υπερ-παραμέτρων για τον τελεστή επιλογής selTournament. Έχουμε δηλαδή συνολικά 32 συνδυασμούς τελεστών. Σε αυτά αντιστοιχούν οι dummy ονομασίες της κολόνας "operators" (πλην του selTournament). Εσείς θα βάλετε τα κανονικά ονόματα των τελεστών με Low και High στο τέλος.



#### Στρατηγική εξέλιξης

Για κάθε συνδυασμό τελεστών θα εξετάσουμε τρεις στρατηγικές: τον **απλό ΓΑ** και τις στρατηγικές εξέλιξης **“μ+λ”** και **“μ,λ”**. Στην κολόνα strategy βλέπετε το όνομα της στρατηγικής και μετά την τιμή ή τις τιμές που χαρακτηρίζουν τον πληθυσμό. Προφανώς o απλός γενετικός έχει μόνο το μέγεθος του πληθυσμού ενώ οι δύο άλλες στρατηγικές έχουν τις τιμές για τα μ και λ. Οι δύο τελευταίοι αριθμοί της κολόνας αντιστοιχούν στις πιθανότητες διασταύρωσης και μετάλλαξης.



#### Μεθοδολογία εύρεσης βέλτιστου συνδυασμού

Αρχικά θα προσδιορίσουμε τους καταλληλότερους συνδυασμούς τελεστών-στρατηγικών. Για το λόγο αυτό θα θέσουμε ένα σχετικά μικρό αριθμό γύρων (π.χ. 5-10) και μέγιστων γενεών (π.χ. 50-100-150) και σταθερές τιμές στις παραμέτρους πληθυσμού και τις πιθανότητες διασταύρωσης και μετάλλαξης.  Διαλέξτε τιμές πληθυσμού έτσι ώστε οι στρατηγικές να έχουν συγκρίσιμο μέσο αριθμό αποτιμήσεων, να μπορούν να σημειωθούν επιτυχείς γύροι και κάθε γύρος να απαιτεί 1.5 - 2 δευτερόλεπτα. Διαλέξτε “λογικές” (μη ακραίες τιμές) για τις πιθανότητες διασταύρωσης και μετάλλαξης (στη βιβλιογραφία επιλέγεται συνήθως μεγαλύτερη πιθανότητα διασταύρωσης και μικρότερη μετάλλαξης).

Διαλέξτε ένα αρχικό DELTA κοντά στο ολικό ελάχιστο του καλύτερου συνδυασμού (από τα πρώτα runs που θα κάνετε). Το “κοντά” σε αυτό το στάδιο μπορούμε να το εκτιμήσουμε μόνο εμπειρικά: επαναλάβετε με διάφορα DELTA έτσι ώστε στην τελική επιλογή DELTA οι καλύτεροι συνδυασμοί να έχουν ποσοστό επιτυχιών κοντά στη μονάδα και οι χειρότεροι καμία. 




#### Τελική αναζήτηση και σχολιασμός

Κρατήστε μόνο την τελική αναζήτηση σε μπλοκ κώδικα και περιγράψτε την πορεία των επιλογών σας σε markdown. 

Σχολιάστε συνολικά τα αποτελέσματα και προτείνετε τους καλύτερους συνδυασμούς τελεστών-στρατηγικών για τη συνάρτησή σας (αιτιολογήστε την επιλογή σας).



### Τελική βελτιστοποίηση



#### Βελτιστοποίηση πιθανοτήτων διασταύρωσης και μετάλλαξης

Για τον καλύτερο συνδυασμό τελεστών-στρατηγικής (αιτιολογήστε την επιλογή σας) και σταθερό αριθμό γύρων, πληθυσμού και γενεών, κάντε grid search στις πιθανότητες των τελεστών διασταύρωσης και μετάλλαξης στο διάστημα [0.05 0.9] ώστε να βρείτε το βέλτιστο συνδυασμό τους. 

Μπορείτε να δοκιμάσετε σταδιακά πιο μικρά διαστήματα περιοχών τιμών (αύξηση της ακρίβειας) γύρω από καλές τιμές.



#### Εύρεση βέλτιστης (ελάχιστης) τιμής της συνάρτησης με τον ΓΑ

Κάντε ένα run του βέλτιστου αλγόριθμου που προέκυψε στα προηγούμενα βήματα με ένα μεγάλο αριθμό γενεών και πληθυσμού για να πάρετε μια βέλτιστη τιμή για τη συνάρτηση (σε λογικά χρονικά πλαίσια). Τυπώστε τη βέλτιστη τιμή, το συνολικό αριθμό αποτιμήσεων και το χρόνο εκτέλεσης.



## Μέρος 2. Μελέτη κλιμακούμενης συνάρτησης

Όπως είδαμε και στο παράδειγμα του 0-1 Knapsack, πολλά προβλήματα μπορεί να λύνονται εύκολα σε μικρές διαστάσεις αλλά γίνονται δυσκολότερα όσο οι διαστάσεις μεγαλώνουν. Συνεπώς, μια επιθυμητή ιδιότητα για έναν αλγόριθμο είναι να μπορεί να αντιμετωπίζει την κλιμάκωση των προβλημάτων. Συνεπώς, η μελέτη της κλιμακούμενης συνάρτησή σας συνίσταται στα ακόλουθα ζητούμενα.



### Για D=2

- α) Εκτυπώστε ένα “3D” γράφημα της συνάρτησης $f(x1,x2)$ και περιγράψτε σύντομα τη μορφή της (βέλτιστα, κοίλα, κοιλάδες, λεκάνες, ασυνέχειες κλπ)
- β) Με την διαδικασία που ακολουθήσαμε προηγουμένως για τη μη κλιμακούμενη συνάρτηση, βρείτε τον βέλτιστο γενετικό αλγόριθμο και τη βέλτιστη τιμή για το πρόβλημα.



### Για D=1, 10, 20, 40 και μεγαλύτερες διαστάσεις

Για τον βέλτιστο αλγόριθμο (που βρήκατε για τις 2 διαστάσεις) και για διαφορετικές τιμές/τάξεις μεγέθους D=1, 10, 20, 40 ή και περισσότερων διαστάσεων του πεδίου ορισμού (σταθερά MAX_GENS και MAX_ROUNDS>=10) τυπώστε πίνακα με: **αριθμό διαστάσεων**, **αριθμό επιτυχιών**, **μέσο ολικό ελάχιστο**, **μέσο αριθμό αποτιμήσεων** και **μέσο χρόνο** (προφανώς βασιστείτε στην αρχική βοηθητική συνάρτηση εκτύπωσης αποτελεσμάτων). 

Σημειώστε ότι οποιεσδήποτε ποινές για άτομα εκτός ορισμού καθώς και τo DELTA, προκειμένου να είναι αποτελεσματικά ως προς την κλιμάκωση των διαστάσεων, θα πρέπει να οριστούν εμπειρικά μεν, αλλά οπωσδήποτε συναρτήσει του πλήθους των διαστάσεων (μεγαλύτερες ποινές για μεγαλύτερες διαστάσεις, για το DELTA έχουν δοθεί οδηγίες σε προηγούμενο section).

α) Σχολιάστε τα αποτελέσματα του πίνακα.

β) Ποιες πιστεύετε είναι οι αιτίες του φαινομένου;

γ) Για σταθερό αριθμό γενεών, πώς μπορούμε να βελτιώσουμε τα αποτελέσματα σε μεγάλες διαστάσεις ως προς το σκέλος της μέσης βέλτιστης τιμής; (Mπορεί να υπάρχουν περισσότερες προσεγγίσεις στο ίδιο πρόβλημα.)

**Σημείωση:** όπως συνήθως συμβαίνει στα υπολογιστικά προβλήματα, μεγαλύτερες διαστάσεις εισόδου σημαίνουν και μεγαλύτερους χρόνους εκτέλεσης. Ταυτόχρονα, οι προς μελέτη συναρτήσεις όλων των ομάδων έχουν διαφορετικές ιδιότητες και βαθμούς δυσκολίας. Συνεπώς, είναι αδύνατον να δώσουμε γενική οδηγία του τί είναι μεγάλες ή μεγαλύτερες διαστάσεις που να ισχύει για όλες τις συναρτήσεις. Δίνουμε κάποιες ενδεικτικές τιμές αλλά μπορείτε να τις προσαρμόσετε (και προς τα κάτω αλλά κυρίως προς τα πάνω) ανάλογα με τους διαθέσιμους πόρους, τον αριθμό επαναλήψεων και του τί θεωρείτε λογικά πλαίσια χρόνου αναμονής.


### Βελτιστοποίηση σε μεγάλες διαστάσεις

Με βάση τις απαντήσεις σας στο πρηγούμενο (β) και πάντοτε με σταθερό αριθμό γενεών, εφαρμόστε τις αλλαγές που χρειάζονται σύμφωνα με την απάντησή σας στο (γ) με τους εξής δύο στόχους:

1. Κρατώντας σταθερή και μεγάλη τη διάσταση εισόδου να παίρνετε όλο και καλύτερες βέλτιστες τιμές της συνάρτησης. Παρουσιάστε τα αποτελέσματα σε συγκριτικό πίνακα παρόμοιο με τον προηγούμενο.

2. Βρείτε μια νέα -σταθερή- διάσταση, πιθανότατα μικρότερη από την προηγούμενη και ένα DELTA που να σας δίνουν 35% - 50% επιτυχίες. Μπορείτε να μειώσετε το DELTA στο μισό και να διπλασιάσετε το ποσοστό των επιτυχιών; Μπορείτε να ξαναμειώσετε στο μισό το DELTA και να αυξήσετε επιπρόσθετα το ποσοστό των επιτυχιών; Αν όχι, κατα πόσο κατ'ελάχιστον πρέπει να αυξήσετε τον αριθμό των γενεών για να πετύχετε σημαντική αύξηση των επιτυχιών;



## Ευρετήρια συναρτήσεων για benchmarking στη βελτιστοποίηση

- [Electric Power Systems Analysis & Nature-Inspired Optimization](https://al-roomi.org/benchmarks/unconstrained)

- [Global Optimization Test Functions Index](http://infinity77.net/global_optimization/test_functions.html)

- [BenchmarkFcns Toolbox](http://benchmarkfcns.xyz/fcns)

- [Virtual Library of Simulation Experiments: Test Functions and Datasets](https://www.sfu.ca/~ssurjano/optimization.html)

- [DEAP Benchmarks functions](http://deap.readthedocs.io/en/master/api/benchmarks.html)




## Βιβλιογραφία

- Jamil, Momin, and Xin-She Yang. "A literature survey of benchmark functions for global optimisation problems." International Journal of Mathematical Modelling and Numerical Optimisation 4.2 (2013): 150-194. [link](https://goo.gl/qAhxNf). Προσοχή στα [errata](https://docs.google.com/document/d/1MrRpq8q_hDfBZVw1CHBzC7t4biQkAftu3LEWXJMf6s0/edit?usp=sharing) στους ορισμούς των συναρτήσεων.

- Whitley, Darrell, et al. "Evaluating evolutionary algorithms." Artificial intelligence 85.1-2 (1996): 245-276. [link](https://www.sciencedirect.com/science/article/pii/0004370295001247/pdf?md5=41088a944920336e5d8493160de27800&pid=1-s2.0-0004370295001247-main.pdf&_valck=1)

- "Gradient Free Optimization". Stanford AA222 - Introduction to Multidisciplinary Design Optimization handouts. [link](http://adl.stanford.edu/aa222/Lecture_Notes_files/chapter6_gradfree.pdf) 






## Σημείωσεις για την υλοποίηση

- Λόγω των πολλών επαναλήψεων είναι απαραίτητη και θα αξιολογηθεί η **εκτεταμένη χρήση συναρτήσεων**.
- Για απορίες και διευκρυνίσεις ως συνήθως συμβουλευτείτε πρώτα το [FAQ](https://docs.google.com/document/d/1jL4gRag_LHbVCYIt5XVJ53iJPb6RZWi02rT5mPXiqEU/edit?usp=sharing) και στη συνέχεια αν χρειαστεί μπορείτε να στείλετε mail στο [nnlab@islab.ntua.gr](mailto:nnlab@islab.ntua.gr)

## Παράδοση

### Ημερομηνία παράδοσης

Πέμπτη 28 Φεβρουαρίου 2019 (παρακαλούμε όχι αιτήματα για παράταση).

### Παραδοτέα

Στο mycourses του μαθήματος εντός ενός zip file:

- το αρχείο ipynb του notebook σας με τις απαντήσεις σας (ορατά τα κελιά εξόδου σύμφωνα με το FAQ). Μην ξεχάσετε ένα κελί με τα ονόματα σας καθώς και τον αριθμό της ομάδας σας.
- ο καθαρός κώδικας Python του notebook δηλαδή το αρχείο .py.


<table>
  <tr>
    <td bgcolor="#FCF8E3"><font size="4">Παρακαλούμε διατρέξτε βήμα-βήμα το notebook για να μην ξεχάσετε παραδοτέα!!</font>
</td>
  </tr>
</table>

**Καλή επιτυχία!** 👍